In [ ]:
!pip install langchain
!pip install langchain_community
!pip install openai
!pip install pinecone-client
!pip install google-cloud-storage
!pip install Python-dotenv
!pip install Firebase-admin
!pip install spacy
!pip install langchain_openai
!pip install pypdf
!pip install pdfminer
!pip install unstructured
!pip install unstructured[pdf]
!pip install docx2txt
!pip install langchainhub
!sudo apt-get install libgl1-mesa-glx

In [ ]:
!pip install Pinecone
!pip install --quiet langchain_experimental langchain_openai

In [ ]:
from operator import le

from langchain.docstore.document import Document
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import SpacyTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader
import os
import spacy
import uuid
import openai
from dotenv import load_dotenv
from langchain_community.document_loaders import OnlinePDFLoader
from langchain_community.vectorstores.pinecone import Pinecone
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.retrieval import create_retrieval_chain

# Conversation imports
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain_community.document_loaders import OnlinePDFLoader, TextLoader
from langchain_community.document_loaders import Docx2txtLoader, PyPDFDirectoryLoader
import shutil
import uuid
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.chains import create_extraction_chain
from typing import Optional, List
from langchain.chains import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel
from langchain import hub
from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
def load_document():
    # use the uploads_dir in your DirectoryLoader
    loader = PyPDFDirectoryLoader("../Documents/", glob="*.pdf")
    docs = loader.load()
    print("docs:", docs)
    return docs

In [ ]:
from pinecone import Pinecone

In [ ]:
def upload_documents_service():
    pinecone = Pinecone(api_key = "42dfe5b5-e733-4640-a90c-16f2a99b2e48")
    index = pinecone.Index("index02")
    docs = load_document()
    for doc in docs:
      print("current document ",doc)
      text = doc.page_content
      embeddings_model = OpenAIEmbeddings(openai_api_key="sk-sEnWhfLa_EQqqYf4tqMfKNj1EJNgTQNbheVCoLJ-QvT3BlbkFJmFyceS00ttsaLH-4PUDo5EBrejSkpCGmLAVE7bnLEA")
      text_splitter = RecursiveCharacterTextSplitter()

      #chunks = text_splitter.split_text(text)

      # chunks = text_splitter.create_documents([text])
      chunks =  text_splitter.create_documents([text])

      # embeddings_arrays = embeddings_model.embed_documents(
      #     [chunk.page_content.replace("\n", " ") for chunk in chunks]
      # )

      batch_size = 100
      batch = []

      for idx in range(len(chunks)):
          chunk = chunks[idx]
          vector = {
              "id": str(uuid.uuid4()),
              "values": embeddings_model.embed_query(
                  chunk.page_content.replace("\n", " ")
              ),
              "metadata": {
                  **chunk.metadata,
                  "text": chunk.page_content,
                  "id": "123",
              },
          }
          batch.append(vector)

          # When batch is full, or it's the last item, upsert the vectors
          if len(batch) == batch_size or idx == len(chunks) - 1:
              index.upsert(vectors=batch)

              # Empty the batch
              batch = []

In [9]:
upload_documents_service()
print("document uploaded")

docs: [Document(metadata={'source': '..\\Documents\\TEST.pdf', 'page': 0}, page_content='It\nis\na\nsample\nfile\nfor\ntesting\nask-ripeseed')]
current document  page_content='It
is
a
sample
file
for
testing
ask-ripeseed' metadata={'source': '..\\Documents\\TEST.pdf', 'page': 0}
document uploaded
